In [1]:
import os
import openpyxl
import locale
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc

load_dotenv()

path_dados = os.getenv('DUSNEI_DATA_DIRECTORY_RELATORIO_PENDENCIAS')

In [2]:
def get_db_engine():
    try:
        db_url = os.getenv('DUSNEI_URL')
        engine = create_engine(db_url)
        # Test connection
        with engine.connect() as connection:
            print('Conexão estabelecida!')
            pass
        print('Banco de dados conectado!')
        return engine
    except exc.SQLAlchemyError as e:
        print(f"Error: {e}")
        return None
        


In [3]:
def vendedores(conn):
    cod_vendedores = []

    query = """
        SELECT
            vend.vend_codigo AS vend_cod,
            vend.vend_nome AS vend_nome
        FROM vendedores AS vend
    """
    df = pd.read_sql_query(query, conn)

    # Adicionando os códigos à lista
    cod_vendedores.extend(df['vend_cod'].tolist())

    return cod_vendedores

In [4]:
def pendencias_clientes_query(conn, codigo_vendedor):
    # Calcula a data de referência
    hoje = datetime.now()
    if hoje.weekday() == 0:  # Segunda-feira
        data_referencia = hoje - timedelta(days=3)  # Sexta-feira
    else:
        data_referencia = hoje - timedelta(days=1)  # Dia anterior

    data_referencia_str = data_referencia.strftime('%Y-%m-%d')
    
    query = (f"""
        SELECT
            pfin.pfin_transacao AS transacao,
            pfin.pfin_status AS status,
            pfin.pfin_datavcto AS datavcto,
            pfin.pfin_pger_conta AS pger_conta,
            pfin.pfin_unid_codigo AS unid_cod,
            pfin.pfin_vend_codigo AS vend_cod,
            pfin.pfin_codentidade AS entidade_cod,
            pfin.pfin_cnpjcpf AS cnpj,
            pfin.pfin_numerodcto AS nota,
            pfin.pfin_parcela AS parcelas,
            pfin.pfin_valor::float AS valor,
            pfin.pfin_nparcelas AS nparcelas,
            pger.pger_descricao AS descricao_contabil,
            pger.pger_tipo AS tipo,
            clie.clie_nome AS clie_nome,
            clie.clie_razaosocial AS razaosocial,
            clie.clie_vend_codigo AS clie_vend_cod,
            clie.clie_foneres AS fone_res,
            clie.clie_fonecel AS fone_cel,
            vend.vend_nome AS vend_nome,
            vend.vend_supe_codigo AS supe_cod,
            CONCAT(vend.vend_extra16, vend.vend_extra15) AS vend_cel,
            unid.unid_reduzido AS unidade,
            supe.supe_nome AS supe_nome,
            muni.muni_nome AS municipio
        FROM pendfin AS pfin
        INNER JOIN planoger AS pger ON pfin.pfin_pger_conta = pger.pger_conta
        INNER JOIN unidades AS unid ON pfin.pfin_unid_codigo = unid.unid_codigo
        INNER JOIN clientes AS clie ON pfin.pfin_codentidade = clie.clie_codigo
        INNER JOIN vendedores AS vend ON clie.clie_vend_codigo = vend.vend_codigo
        INNER JOIN supervisores AS supe ON vend.vend_supe_codigo = supe.supe_codigo
        INNER JOIN municipios AS muni ON clie.clie_muni_codigo_res = muni.muni_codigo
        WHERE pfin.pfin_status = 'P'
        AND pfin.pfin_datavcto < CURRENT_DATE
        AND pfin.pfin_vend_codigo = '{codigo_vendedor}'
        AND pfin.pfin_pger_conta NOT IN ('211851','112701','112801','211906','112125','112101','211810','211825','113301','211127','211102','112951','211812')
        ORDER BY pfin.pfin_pger_conta ASC
    """)
    return pd.read_sql_query(query, conn)

In [5]:
def gerar_excel(df, ws, codigo_vendedor):
    ws.title = f"Vendedor {codigo_vendedor}"
    
    columns_title = [
        'CodClie','RazaoSocial','Nota','Valor','Vencimento','Supervisor','Vendedor','C.Descrição'
    ]
    
    row_title = 1

    for col_num, column_title in enumerate(columns_title, 1):
        cell = ws.cell(row=row_title, column=col_num)
        cell.value = column_title

    for index, row in enumerate(df.itertuples(), start=2):
        ws.cell(row=index, column=1).value = row.entidade_cod
        ws.cell(row=index, column=2).value = row.razaosocial
        ws.cell(row=index, column=3).value = row.nota
        ws.cell(row=index, column=4).value = row.valor
        ws.cell(row=index, column=4).number_format = 'R$ #,##0.00'
        ws.cell(row=index, column=5).value = row.datavcto.strftime("%Y/%m/%d")
        ws.cell(row=index, column=6).value = row.supe_nome
        ws.cell(row=index, column=7).value = row.vend_nome
        ws.cell(row=index, column=8).value = row.descricao_contabil


In [6]:
def gerar_excel_resumo(df, wb):
    resumo_df = df.groupby('descricao_contabil', as_index=False)['valor'].sum()
      
    ws_resumo = wb.create_sheet(title="Resumo Contas")
    
    columns_title = [
        'Descrição','Valor'
    ]

    row_title = 1

    for col_num, column_title in enumerate(columns_title, 1):
        ws_resumo.cell(row=row_title, column=col_num).value = column_title

    for index, row in enumerate(resumo_df.itertuples(), start=2):
        ws_resumo.cell(row=index, column=1).value = row.descricao_contabil
        ws_resumo.cell(row=index, column=2).value = row.valor
        # Define o estilo de número da célula para ter duas casas decimais
        ws_resumo.cell(row=index, column=2).number_format = 'R$ #,##0.00'


In [7]:
def gerar_excel_cidade(df, wb):
    # Agrupar por 'vend_cod' e somar os valores
    resumo_df = df.groupby('municipio', as_index=False)['valor'].sum()

    # Criar uma nova aba no workbook
    ws_resumo = wb.create_sheet(title="Resumo Municipio")

    # Títulos das colunas
    columns_title = ['Municipio', 'Valor']
    row_title = 1

    # Preencher títulos das colunas
    for col_num, column_title in enumerate(columns_title, 1):
        ws_resumo.cell(row=row_title, column=col_num).value = column_title

    # Preencher dados nas células
    for index, row in enumerate(resumo_df.itertuples(), start=2):
        ws_resumo.cell(row=index, column=1).value = row.municipio
        # ws_resumo.cell(row=index, column=2).value = f"{row.valor:.2f}".replace(".",",")
        ws_resumo.cell(row=index, column=2).value = row.valor
        # Define o estilo de número da célula para ter duas casas decimais
        ws_resumo.cell(row=index, column=2).number_format = 'R$ #,##0.00'

In [8]:
def teste_loop():
    conn = get_db_engine()
    lista_vendedor = vendedores(conn)

    for codigo_vendedor in lista_vendedor:
        df = pendencias_clientes_query(conn, codigo_vendedor)
        
        if df.empty:
            print(f"Não há dados para o vendedor {codigo_vendedor}. Arquivo não será gerado.")
            continue  # Pula para o próximo vendedor

    
        locale.setlocale(locale.LC_MONETARY, 'pt-BR.UTF-8')
        wb = openpyxl.Workbook()
        ws = wb.active
        
        gerar_excel(df, ws, codigo_vendedor)
        gerar_excel_resumo(df, wb)
        gerar_excel_cidade(df, wb)

        data_pasta = datetime.now().strftime("%Y-%m-%d")
        nome_arquivo = f'{codigo_vendedor}-pendencias-{data_pasta}'
        diretorio = os.path.join(path_dados, data_pasta, 'vendedores')

        if not os.path.exists(diretorio):
            os.makedirs(diretorio, exist_ok=True)

        local_arquivo = os.path.join(diretorio, f'{nome_arquivo}.xlsx')
        wb.save(local_arquivo)
    
    print("Arquivos criados")


teste_loop()

Conexão estabelecida!
Banco de dados conectado!
Não há dados para o vendedor 124. Arquivo não será gerado.
Não há dados para o vendedor 117. Arquivo não será gerado.
Não há dados para o vendedor 139. Arquivo não será gerado.
Não há dados para o vendedor 091. Arquivo não será gerado.
Não há dados para o vendedor 112. Arquivo não será gerado.
Não há dados para o vendedor 048. Arquivo não será gerado.
Não há dados para o vendedor 064. Arquivo não será gerado.
Não há dados para o vendedor 130. Arquivo não será gerado.
Não há dados para o vendedor 012. Arquivo não será gerado.
Não há dados para o vendedor 122. Arquivo não será gerado.
Não há dados para o vendedor 057. Arquivo não será gerado.
Não há dados para o vendedor 040. Arquivo não será gerado.
Não há dados para o vendedor 035. Arquivo não será gerado.
Não há dados para o vendedor 147. Arquivo não será gerado.
Não há dados para o vendedor 137. Arquivo não será gerado.
Não há dados para o vendedor 066. Arquivo não será gerado.
Não há d